In [1]:
import re
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
import matplotlib.path as path

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
with open('resources/muckrake-report.html', 'r') as f:
    contents = f.read()

contents = contents.replace("\n", " ")
contents = contents.replace("\\n", " ")


IOError: [Errno 2] No such file or directory: 'resources/muckrake-report.html'

In [ ]:
def extract_objects(string):
    obj_list = []
    stack = []
    obj = ""
    for i, c in enumerate(string):
        if c == '{':
            stack.append(c)
            obj += c
        elif c == '}':
            assert len(stack) > 0
            stack.pop()
            obj += c
            if len(stack) == 0:
                obj_list.append(obj)
                obj = ""
        elif len(stack) == 0:
            # ignore characters outside of curly brackets
            pass
        else:
            obj = obj + c
            
    return obj_list

In [ ]:
m = re.search(".+TESTS=(\[.*\])", contents)
s = m.groups()[0]

In [3]:
object_strings = extract_objects(s)
len(object_strings)
for o in object_strings:
    print o

NameError: name 'extract_objects' is not defined

In [6]:
failed = []
raw_tests = []
for o in object_strings:
    try:
        raw_tests.append(json.loads(o))
    except ValueError as e:
        failed.append(o)
assert len(failed) == 0

In [5]:
def parse_run_time_string(rs):
    x = rs.split(' ')
    assert len(x) == 2 or len(x) == 4
    if len(x) == 4:
        minutes = int(x[0])
        seconds = int(float(x[2]))
    else:
        minutes = 0
        seconds = int(float(x[0]))
        
    total_seconds = 60 * minutes + seconds
    
    # round to nearest 5 seconds
    total_seconds = 5 * (int(total_seconds / 5))
    return total_seconds

In [126]:
#[parse_run_time_string(r['run_time']) for r in raw_tests ]

In [127]:
def transform_test(test):
    return { 
        'name': test['test_name'],
        'run_time_seconds': parse_run_time_string(test['run_time'])
    }

In [129]:
parsed_test_summaries = [transform_test(r) for r in raw_tests]

In [137]:
with open('muckrake-tests.json', 'w') as f:
    json.dump(parsed_test_summaries, f, sort_keys=True, indent=4, separators=(',', ': '))

In [7]:
with open('resources/muckrake.json', 'r') as f:
    all_tests = json.load(f)

In [ ]:
np.array([x['run_time_seconds'] for x in all_tests])

In [ ]:
fig, ax = plt.subplots()

# histogram our data with numpy
data = np.array([x['nodes'] for x in all_tests])
n, bins = np.histogram(data, 50)

# get the corners of the rectangles for the histogram
left = np.array(bins[:-1])
right = np.array(bins[1:])
bottom = np.zeros(len(left))
top = bottom + n


# we need a (numrects x numsides x 2) numpy array for the path helper
# function to build a compound path
XY = np.array([[left, left, right, right], [bottom, top, top, bottom]]).T

# get the Path object
barpath = path.Path.make_compound_path_from_polys(XY)

# make a patch out of it
patch = patches.PathPatch(
    barpath, facecolor='blue', edgecolor='gray', alpha=0.8)
ax.add_patch(patch)

# update the view limits
ax.set_xlim(left[0], right[-1])
ax.set_ylim(bottom.min(), top.max())

plt.show()